### 정규 표현식 활용

### (1) HTML 문서에서 URL Link 추출하기

In [9]:
import re

s = '''<a href="http://www.daum.net">daum</a>  
<a href='http://www.naver.com'>naver</a>  
<a href=http://www.chosun.com>chosun</a>  
<a href=http://www.chosun.com class>chosun</a>  
<a href="http://job.daum.net/ " target="new">  
<a href="http://go.daum.net/bin/go.cgi?relative=1&url=/Mail-bin/login_f.cgi%3Ferror%3Dlogin" class="tls0">  
'''  

# (1)'<a href=' 뒤에 큰 따옴표(" ") 가 오는 경우
p = re.compile('''href="([^"]*?)"''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.daum.net']

# (2) '<a href='뒤에 작은 따옴표(' ')가 오는 경우
p = re.compile('''href='([^']*?)\'''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.naver.com']

# (3) '<a href='뒤에 큰/작은 따옴표가 안나오고 마지막 공백이 오는 경우
p = re.compile('''href=([^'"]\S+?)[\s>]''', re.I)
match = p.search(s,0)
url = match.groups()   #  매칭된 전체 그룸 문자열을 튜플 형식으로 반환
# print(url)
print(list(filter(None,url)))   # ['http://www.chosun.com']

['http://www.daum.net']
['http://www.naver.com']
['http://www.chosun.com']


In [2]:
# 세개의 정규표현식을 '|'를 사용하여 하나로 표현 한 경우
p = re.compile('''href="([^"]*?)"|href='([^']*?)\'|href=([^'"]\S+?)[\s>]''')
pos = 0
i = 0
while True:
    #print(i,pos)
    i += 1
    match = p.search(s,pos)
    if match:
        url = match.groups()
        #print(url)
        pos = match.end()
        print(list(filter(None,url))[0])  # 리스트의 첫번째  요소접근
    else:
        break

# filter함수는 특정 조건으로 걸러서 걸러진 요소들로 iterator객체를 만들어서 리턴해줍니다.
# map함수와 사용 방법은 동일하나, 함수의 결과가 참인지 거짓인지에 따라, 해당 요소를 포함할지를 결정합니다.      
 

http://www.daum.net
http://www.naver.com
http://www.chosun.com
http://www.chosun.com
http://job.daum.net/ 
http://go.daum.net/bin/go.cgi?relative=1&url=/Mail-bin/login_f.cgi%3Ferror%3Dlogin


In [3]:
# filter 함수 사용법 설명
print(list(filter(None,(None,10,None))))
print(list(filter(None,(1,None,10))))
print(list(filter(None,(None,None,None))))

[10]
[1, 10]
[]


In [4]:
L = ['apple','banana','','grape']
list(filter(None,L)) # 리스트에서 의미 없는 값을 삭제할 대 사용
 # filter() 함수의 첫번째 인수로 None 객체를 사용하면 입력값을 진리값을 판별하는데 그대로 사용할 수 있다

['apple', 'banana', 'grape']

In [5]:
# filter함수의 필요성
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = []

def is_even(n):
      return True if n % 2 == 0 else False

for value in target:
    print(is_even(value))
        
    if is_even(value):
        result.append(value)

print(result)  # 출력결과 : [2, 4, 6, 8, 10]

False
True
False
True
False
True
False
True
False
True
[2, 4, 6, 8, 10]


In [6]:
# filter 함수 사용
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

def is_even(n):
    return True if n % 2 == 0 else False

result = filter(is_even, target)

print(result,list(result)) # 출력결과 : [2, 4, 6, 8, 10]

<filter object at 0x000001FA5D6BCE08> [2, 4, 6, 8, 10]


### (2) 어휘 분석기 구현

In [38]:
# 파이썬 소스코드를 키워드별로 분석해준다

import re
import keyword

# print('|'.join(keyword.kwlist))

# 정규식 리스트
tokenREList = [
    ('KEYWORD','|'.join(keyword.kwlist)), # 파이썬 키워드
    ('NAME',r'[a-zA-Z_]\w*'),             # 첫문자는 영문자나 밑줄로 시작,두번째부터는 문자는 영문자나 숫자나 밑줄로 시작,키워드가 아닌 변수나 함수
    ('NUMBER', r'\d+|\d+\.\d*|\.\d+'),    # 정수나 실수(소수점 사용), 정수 | 정수.정수 |.정수
    ('LPAREN', r'\('),                    # 왼쪽 인자
    ('RPAREN', r'\)'),                    # 오른쪽 인자
    ('PLUS', r'\+'),                      # 더하기
    ('COLON', r':'),                      # 콜론
    ('WHITESPACE', r'\s+')                # 공백
]

rexp = ['(%s)'%exp for tokName, exp in tokenREList ]
# print(rexp)

p = re.compile('|'.join(rexp))

def lex(s):
    pos = 0
    match = p.match(s,pos)
    i = 0
    while match:
        #print(i)
        i += 1
        
        token = match.groups()
        #print(token)
        
        pos = match.end()
        #print(pos)
        for k in range(len(token)) : # 8회
            if token[k] :
                break
        yield (tokenREList[k][0], token[k])
        match = p.match(s,pos)

if __name__ == '__main__':
    s = '''
    for i in range(10):
        print (i + 5)''' 
    for tok in lex(s): #  22회
        print(tok)
    

('WHITESPACE', '\n    ')
('KEYWORD', 'for')
('WHITESPACE', ' ')
('NAME', 'i')
('WHITESPACE', ' ')
('KEYWORD', 'in')
('WHITESPACE', ' ')
('NAME', 'range')
('LPAREN', '(')
('NUMBER', '10')
('RPAREN', ')')
('COLON', ':')
('WHITESPACE', '\n        ')
('NAME', 'print')
('WHITESPACE', ' ')
('LPAREN', '(')
('NAME', 'i')
('WHITESPACE', ' ')
('PLUS', '+')
('WHITESPACE', ' ')
('NUMBER', '5')
('RPAREN', ')')


### 제네레이터(Generator , 발생자)

In [14]:
# 기존의 함수 호출 방식은 함수가 호출될 때 인수들의 내부 변수들이 새로운 영역(스택)에 
# 만들어지고 반환시 메모리에서 소멸된다
def f(a,b):
    c = a * b
    d = a + b
    return c,d

print(f(10,20))
print(f(1,2))

(200, 30)
(2, 3)


In [17]:
# 제네레이터는 중단된 시점부터 재 실행이 가능한 함수
# return 대신에 키워드 yield를 사용한다

def gen_func():
    print('1st step')
    yield 1
    print('2nd step')
    yield 2
    print('3nd step')
    yield 3  
    
g = gen_func()  # generator 객체를 생성한다
print(type(g))

a = next(g)   # 최초 호출
a

<class 'generator'>
1st step


1

In [18]:
b = next(g)
b

2nd step


2

In [19]:
c = next(g)
c

3nd step


3

In [22]:
# genertor를 for 문으로 호출
def generator_ints(N):
    for k in range(N):
        yield k
        
for k in generator_ints(10):
    print(k,end=' ')

print(type(generator_ints(10))) #  제네레이터는 반복자 객체이다 

0 1 2 3 4 5 6 7 8 9 <class 'generator'>


### 코루틴(Coroutine)
: 함수 실행에 있어서 어떤 위치에서 중단과 실행이 가능한 다중 진입점이 있는 일반화된 함수를 말한다

In [23]:
def echo():
    print('echo routine')
    while True:
        msg = (yield)
        print('echo:',msg)

In [24]:
e = echo() # 코루틴 객체를 생성
print(type(e))   # <class 'generator'>
next(e)

<class 'generator'>
echo routine


In [25]:
next(e)

echo: None


In [26]:
e.send('Hello~')

echo: Hello~


In [27]:
e.send('Bye~')
e.close()

echo: Bye~


In [30]:
# 양방향 값 전송 코루틴
def accumulate(value = 0):
    acc = value
    while True:
        value = (yield acc,value)
        acc += value
        
acc = accumulate(1) 
print(type(acc))
next(acc)

<class 'generator'>


(1, 1)

In [31]:
acc.send(2)   # 2를 value에 전달해주고 다음 yield문 까지 진행 ,(3,2)를 반환 받는다

(3, 2)

In [32]:
acc.send(3)   # 3를 value에 전달해주고 다음 yield문 까지 진행 ,(6,3)를 반환 받는다

(6, 3)

In [33]:
acc.close()   # 코루틴을 종료한다

### (3) 미니 챗봇 구현

In [44]:
r = "(hi|hello|hey)[ ]*([a-z]*)"
re.match(r,'Hello Rosa',re.I)

<re.Match object; span=(0, 10), match='Hello Rosa'>

In [47]:
re.match(r,"hi ho, hi hoho, it's off to work...",re.I)

<re.Match object; span=(0, 5), match='hi ho'>

In [48]:
re.match(r,"hey, what's up",re.I)

<re.Match object; span=(0, 3), match='hey'>

In [49]:
# 정규 표현식을 더 확장
r = r"[^a-z]*([y]o|[h]?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r,re.I)

In [73]:
re_greeting.match('Hello Rosa')

<re.Match object; span=(0, 10), match='Hello Rosa'>

In [64]:
re_greeting.match('Good morning Rosa')

<re.Match object; span=(0, 17), match='Good morning Rosa'>

In [65]:
re_greeting.match('Good marning Rosa')  # 실패

In [66]:
re_greeting.match('Good evening Rosa Parks').groups()

('Good evening', 'Good ', 'evening', 'Rosa')

In [68]:
re_greeting.match("Good morn'n Rosa")

<re.Match object; span=(0, 16), match="Good morn'n Rosa">

In [69]:
re_greeting.match("yo Rosa")

<re.Match object; span=(0, 7), match='yo Rosa'>

In [79]:
# Chatbot
r = r"[^a-z]*([y]o|[h]?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r,re.I)

my_names = set(['rosa','rose','chatty','chatbot','bot','chatterbot','baby']) # 좋은 단어
curt_names = set(['hal','you','u','guy','boy']) # 무뚜뚝한 단어
greeter_name = '길동' # 대화 상대자 (사람)

match = re_greeting.match(input())  # 'Hello Rosa'

if match:
    at_name = match.groups()[-1] # 4개 그룹중 마지막 그룹
    if at_name in curt_names:
        print('Um.. Have a good time.')
    elif at_name.lower() in my_names:
        print('Hi {}, How are you? Glad to see you.'.format(greeter_name))        

Good morning Baby
Hi 길동, How are you? Glad to see you.


In [80]:
# 실습 : 챗봇 소스에 기능 추가
# evil_names 목록을 넣어 대화에 사용하면 화를 내는 코드를 간단히 추가하고 동작을 확인 해보세요
# 영어 대답 아래에 동일한 한국어 대답도 출력하게 해보세요
# while True를 사용하여 match가 없을 때 종료하도록 수정하세요
# evil_names : set(['idiot','fool','ass','sob'])

대화가 시작되었습니다!
----------------------------------------------------------------------
길동 : Hello Rosa
컴퓨터 : Hi 길동, How are you? Glad to see you.
컴퓨터 : 길동씨 안녕하세요,만나서 반갑습니다
----------------------------------------------------------------------
길동 : Hi you
컴퓨터 : Um.. Have a good time.
컴퓨터 : 음.. 즐거운 시간 되세요.
----------------------------------------------------------------------
길동 : Hey idiot
컴퓨터 : Dear 길동 please, don't talk to me like that : idiot. 
컴퓨터 : 길동아! 제발 그런말 쓰지마! : idiot. 
----------------------------------------------------------------------
길동 : Bye
대화가 종료되었습니다!

In [81]:
# 실습 답안 : 챗봇 기능 추가 답안

import re

r = r"[^a-z]*([y]o|[h]?ello|ok|hi|hey|(good[ ])?(morn[gin']{0,3}|afternoon|even[gin']{0,3}))[\s,:;]{1,3}([a-z]{1,20})"                                            
re_greeting = re.compile(r, re.I)
my_names = set(['rosa','rose','chatty','chatbot','bot','chatterbot','baby']) # 좋은 단어들 리스트
curt_names = set(['hal','you','u','guy','boy']) # curt(무뚜뚝한) 이름, 다소 무례한 단어들 리스트
evil_names = set(['idiot','fool','ass','sob'])  # 나쁜 단어들 리스트
greeter_name = '길동'  # 대화 상대자(사람)
computer = '컴퓨터 :'
# match = re_greeting.match(input(greeter_name+' : '))
print('대화가 시작되었습니다!')
print('-'*70)

while True:
    match = re_greeting.match(input(greeter_name+' : '))
    # print(match)
    if match :
        at_name = match.groups()[-1]
        # print(at_name)
        if at_name in curt_names:
            print(computer,"Um.. Have a good time.") 
            print(computer,"음.. 즐거운 시간 되세요.")
        elif at_name.lower() in my_names:
            print(computer,"Hi {}, How are you? Glad to see you.".format(greeter_name))
            print(computer,"{}씨 안녕하세요,만나서 반갑습니다".format(greeter_name))
        elif at_name in evil_names:
            print(computer,"Dear {0} please, don't talk to me like that : {1}. ".format(greeter_name,at_name))
            print(computer,"{0}아! 제발 그런말 쓰지마! : {1}. ".format(greeter_name,at_name))
        print('-'*70)
    else:
        break
    
print('대화가 종료되었습니다!')    

대화가 시작되었습니다!
----------------------------------------------------------------------
길동 : Hello Rosa
컴퓨터 : Hi 길동, How are you? Glad to see you.
컴퓨터 : 길동씨 안녕하세요,만나서 반갑습니다
----------------------------------------------------------------------
길동 : Hey guy
컴퓨터 : Um.. Have a good time.
컴퓨터 : 음.. 즐거운 시간 되세요.
----------------------------------------------------------------------
길동 : Hey idiot
컴퓨터 : Dear 길동 please, don't talk to me like that : idiot. 
컴퓨터 : 길동아! 제발 그런말 쓰지마! : idiot. 
----------------------------------------------------------------------
길동 : Bye
대화가 종료되었습니다!
